# polarization fraction tests

[L. Blackburn, Sep 2018]

The purpose of this test is to check stability of polarization fraction on various baselines. The test is sensitive to leakage and false fringes.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
from collections import OrderedDict
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
from matplotlib.ticker import MultipleLocator
import seaborn as sns
sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']
import sys
nb_stdout = sys.stdout # grab for later
def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()
def tightx(): plt.autoscale(enable=True, axis='x', tight=True)
def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')
def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])
# pd.options.display.float_format = '{:,.6f}'.format
from IPython.core.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))
# Run this in Python once, it should take effect permanently
from notebook.services.config import ConfigManager
c = ConfigManager()
_ = c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
import string

In [ ]:
os.getcwd()

In [ ]:
# define and load data
alistf = 'alist.v6'
# stage = '2.+pcal'
# datadir = '/data/2017-april/ce/er4test2/hops-lo/%s/data/' % stage
datadir = os.environ['DATADIR'] + '/'
a = util.noauto(hops.read_alist(datadir + alistf))
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)
util.add_gmst(a)
days = sorted(set(a.expt_no))

In [ ]:
# data filters
a = a[(a.snr > 7.) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# expt_no boundary ticker marks vs scan
elines = a.scan_no.sort_values().values[np.nonzero(np.diff(a.expt_no.sort_values()) > 0)[0]] + 0.5

In [ ]:
# show all output in cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
p = a.pivot_table(aggfunc='first', index="expt_no scan_no gmst timetag baseline source u v".split(),
    columns=['polarization'], values=['snr']).dropna()
p['fpol'] = np.sqrt((p.snr.LR * p.snr.RL) / (p.snr.LL * p.snr.RR))
p['fpol_err'] = np.sqrt(2. / (p.snr.LL * p.snr.RR))
q = p.reset_index()

In [ ]:
def pftrend(b, src):
    d = b[b.source == src].copy()
    t = np.hstack((d.gmst.sort_values().values, d.gmst.sort_values().values + 24.))
    idx = np.argmax(np.diff(t))
    toff = np.fmod(48. - 0.5 * (t[idx] + t[1+idx]), 24.)
    d['gmst'] = np.fmod(d.gmst + toff, 24.) - toff
    if d.gmst.max() < 0:
        d.gmst += 24
    ax = None
    blc = dict(zip(sorted(set(d.baseline)),
                   itertools.cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])))
    lnum = 1e6
    for(i, day) in enumerate(days):
        ax = plt.subplot(1, len(days), 1+i, sharey=ax, sharex=ax)
        dayrows = d[d.expt_no == day]
        if len(dayrows) < lnum:
            (lax, lnum) = (ax, len(dayrows))
        if i > 0:
            _ = plt.setp(ax.get_yticklabels(), visible=False)
        for (bl, blrows) in dayrows.groupby('baseline'):
            h = plt.errorbar(blrows.gmst, blrows.fpol, blrows.fpol_err, fmt='.', color=blc[bl], label='_nolegend_')
            _ = plt.plot(blrows.gmst, blrows.fpol, '-', color=h[0].get_color(), alpha=0.25, label='_nolegend_')
        ax.grid(axis='y', alpha=0.25)
    lines = [plt.Line2D([0], [0], color=blc[bl], marker='.', ls='none') for bl in sorted(blc.keys())]
    leg = Legend(lax, lines, sorted(blc.keys()), loc='best', ncol=2)
    _ = lax.add_artist(leg)
    wide(12, 4)
    plt.subplots_adjust(hspace=0, wspace=0)
    _ = plt.suptitle('%s fractional polarization vs GMST' % src, y=plt.gcf().subplotpars.top, va='bottom')

In [ ]:
def pfuv(b, src, cmap='jet'):
    d = b[b.source == src].sort_values('fpol')
    ax = None
    lim = 1.1e-3 * max(d.u.abs().max(), d.v.abs().max())
    for(i, day) in enumerate(days):
        ax = plt.subplot(1, len(days), 1+i, sharey=ax, sharex=ax, aspect=1.0)
        dayrows = d[d.expt_no == day]
        if i > 0:
            _ = plt.setp(ax.get_yticklabels(), visible=False)
        _ = plt.scatter(1e-3 * dayrows.u, 1e-3 * dayrows.v, c=dayrows.fpol, cmap=cmap, vmin=0, vmax=1)
        _ = plt.scatter(-1e-3 * dayrows.u, -1e-3 * dayrows.v, c=dayrows.fpol, cmap=cmap, vmin=0, vmax=1)
        _ = plt.grid(which='both', ls='--', alpha=0.25)
    wide(12, 3)
    plt.xlim(-lim, lim)
    plt.ylim(-lim, lim)
    plt.subplots_adjust(hspace=0, wspace=0)
    _ = plt.suptitle('%s fractional polarization vs (u, v) [Gly]' %
                     src, y=plt.gcf().subplotpars.top, va='bottom')

In [ ]:
from matplotlib.legend import Legend
for src in sorted(set(q.source)):
    pftrend(q, src)
    plt.show()
    pfuv(q, src, 'jet')
    plt.show()